**[Natural Language Processing Home Page](https://www.kaggle.com/learn/natural-language-processing)**

---


# Basic Text Processing with Spacy
    
You're a consultant for [DelFalco's Italian Restaurant](https://defalcosdeli.com/index.html).
The owner asked you to identify whether there are any foods on their menu that diners find disappointing. 

<img src="https://i.imgur.com/8DZunAQ.jpg" alt="Meatball Sub" width="250"/>

Before getting started, run the following cell to set up code checking.

In [1]:
# setup code checking
from learntools.core import binder
binder.bind(globals())
from learntools.nlp.ex1 import *
print('Setup is completed.')

Setup is completed.


The business owner suggested you use diner reviews from the Yelp website to determine which dishes people liked and disliked. You pulled the data from Yelp. Before you get to analysis, run the code cell below for a quick look at the data you have to work with.

In [2]:
# load in the data from JSON file
import pandas as pd
data = pd.read_json('../input/nlp-course/restaurant.json')
data.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
109,lDJIaF4eYRF4F7g6Zb9euw,lb0QUR5bc4O-Am4hNq9ZGg,r5PLDU-4mSbde5XekTXSCA,4,2,0,0,I used to work food service and my manager at ...,2013-01-27 17:54:54
1013,vvIzf3pr8lTqE_AOsxmgaA,MAmijW4ooUzujkufYYLMeQ,r5PLDU-4mSbde5XekTXSCA,4,0,0,0,We have been trying Eggplant sandwiches all ov...,2015-04-15 04:50:56
1204,UF-JqzMczZ8vvp_4tPK3bQ,slfi6gf_qEYTXy90Sw93sg,r5PLDU-4mSbde5XekTXSCA,5,1,0,0,Amazing Steak and Cheese... Better than any Ph...,2011-03-20 00:57:45
1251,geUJGrKhXynxDC2uvERsLw,N_-UepOzAsuDQwOUtfRFGw,r5PLDU-4mSbde5XekTXSCA,1,0,0,0,Although I have been going to DeFalco's for ye...,2018-07-17 01:48:23
1354,aPctXPeZW3kDq36TRm-CqA,139hD7gkZVzSvSzDPwhNNw,r5PLDU-4mSbde5XekTXSCA,2,0,0,0,"Highs: Ambience, value, pizza and deserts. Thi...",2018-01-21 10:52:58


The owner also gave you this list of menu items and common alternate spellings.

In [3]:
menu = [
    "Cheese Steak", "Cheesesteak", "Steak and Cheese", "Italian Combo", "Tiramisu", "Cannoli",
    "Chicken Salad", "Chicken Spinach Salad", "Meatball", "Pizza", "Pizzas", "Spaghetti",
    "Bruchetta", "Eggplant", "Italian Beef", "Purista", "Pasta", "Calzones",  "Calzone",
    "Italian Sausage", "Chicken Cutlet", "Chicken Parm", "Chicken Parmesan", "Gnocchi",
    "Chicken Pesto", "Turkey Sandwich", "Turkey Breast", "Ziti", "Portobello", "Reuben",
    "Mozzarella Caprese",  "Corned Beef", "Garlic Bread", "Pastrami", "Roast Beef",
    "Tuna Salad", "Lasagna", "Artichoke Salad", "Fettuccini Alfredo", "Chicken Parmigiana",
    "Grilled Veggie", "Grilled Veggies", "Grilled Vegetable", "Mac and Cheese", "Macaroni",  
    "Prosciutto", "Salami"
]

<hr/>

# Step 1: Plan Your Analysis

Given the data from Yelp and the list of menu items, do you have any ideas for how you could find which menu items have disappointed diners?

Think about your answer. Then run the cell below to see one approach.

In [4]:
# check your answer (run this code cell to receive credit!)
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> You could group reviews by what menu items they mention, and then calculate the average rating
    for reviews that mentioned each item. You can tell which foods are mentioned in reviews with low scores,
    so the restaurant can fix the recipe or remove those foods from the menu.

<hr/>

# Step 2: Find Items in One Review

You'll pursue this plan of calculating average scores of the reviews mentioning each menu item.

As a first step, you'll write code to extract the foods mentioned in a single review.

Since menu items are multiple tokens long, you'll use `PhraseMatcher` which can match series of tokens.

Fill in the `____` values below to get a list of items matching a single menu item.

In [5]:
index_of_review_to_test_on = 14
text_to_test_on = data['text'].iloc[index_of_review_to_test_on]

# create a blank model of a given language class. this function is the twin of `spacy.load()`
import spacy
nlp = spacy.blank('en')

# create the tokenized version of text_to_test_on
review_doc = nlp(text_to_test_on)

# create the PhraseMatcher object, to match a list of terms
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(
    nlp.vocab,          # tokenizer
    attr='LOWER'        # to make consistent capitalization
)

# create a list of tokens for each item in the menu
menu_tokens_list = [nlp(item) for item in menu]

# add the item patterns to the matcher, for more help: https://spacy.io/api/phrasematcher#add
matcher.add(
    "MENU",            # Just a name for the set of rules we're matching to
    None,              # Special actions to take on matched words
    *menu_tokens_list  
)

# find matches in the review_doc
matches = matcher(review_doc)

# check your work
q_2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# lines below will give you a hint or solution code
# q_2.hint()
# q_2.solution()

After implementing the above cell, run the following cell to print the matches.

In [7]:
print(review_doc)
print('-'*10)
print(matches)
print('-'*10)
for match in matches:
   print(f"Token number {match[1]}: {review_doc[match[1]:match[2]]}")

The Il Purista sandwich has become a staple of my life. Mozzarella, basil, prosciutto, roasted red peppers and balsamic vinaigrette blend into a front runner for the best sandwich in the valley. Goes great with sparkling water or a beer. 

DeFalco's also has other Italian fare such as a delicious meatball sub and classic pastas.
----------
[(8291075388056826051, 2, 3), (8291075388056826051, 16, 17), (8291075388056826051, 58, 59)]
----------
Token number 2: Purista
Token number 16: prosciutto
Token number 58: meatball


<hr/>

# Step 3: Matching on the Whole Dataset

Now run this matcher over the whole dataset and collect ratings for each menu item. Each review has a rating, `review['stars']`. For each item that appears in the review text (`review['text']`), append the review's rating to a list of ratings for that item. The lists are kept in a dictionary `item_ratings`.

To get the matched phrases, you can reference the `PhraseMatcher` documentation for the structure of each match object:

>A list of `(match_id, start, end)` tuples, describing the matches. A match tuple describes a span `doc[start:end]`. The `match_id` is the ID of the added match pattern.

In [8]:
from collections import defaultdict

# item_ratings is a dictionary of lists. If a key doesn't exist in item_ratings,
# the key is added with an empty list as the value.
item_ratings = defaultdict(list)

for idx, review in data.iterrows():
    doc = nlp(review['text'])
    
    # using the matcher from the previous exercise
    matches = matcher(doc)
    
    # create a set of the items found in the review text
    found_items = set([doc[match[1]:match[2]] for match in matches])
    
    # Update item_ratings with rating for each item in found_items
    # Transform the item strings to lowercase to make it case insensitive
    for item in found_items:
        item_ratings[str(item).lower()].append(review['stars'])

# check your work
q_3.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [9]:
# lines below will give you a hint or solution code
# q_3.hint()
# q_3.solution()

<hr/>

# Step 4: What's the worst reviewed item?

Using these item ratings, find the menu item with the worst average rating.

In [10]:
# calculate the mean ratings for each menu item as a dictionary
from statistics import mean
mean_ratings = {item: mean(ratings) for item, ratings in item_ratings.items()}

# find the worst item, and write it as a string in worst_text
worst_item = min(mean_ratings, key=mean_ratings.get)

# check your work
q_4.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [11]:
# lines below will give you a hint or solution code
# q_4.hint()
# q_4.solution()

In [12]:
# print out the worst item, along with its average rating. 
print(worst_item)
print(mean_ratings[worst_item])

chicken cutlet
3.5454545454545454


<hr/>

# Step 5: Are counts important here?

Similar to the mean ratings, you can calculate the number of reviews for each item.

In [13]:
counts = {item: len(ratings) for item, ratings in item_ratings.items()}

item_counts = sorted(counts, key=counts.get, reverse=True)
for item in item_counts:
    print(f"{item:>25}{counts[item]:>5}")

                    pizza  358
                    pasta  255
                 meatball  163
              cheesesteak  146
                  calzone  110
                 eggplant   95
                  cannoli   89
             cheese steak   88
                  lasagna   83
                  purista   67
               prosciutto   63
             chicken parm   58
          italian sausage   57
             garlic bread   46
                  gnocchi   45
                spaghetti   41
                 calzones   38
                   pizzas   33
                   salami   32
            chicken pesto   30
             italian beef   29
                 tiramisu   27
                     ziti   26
            italian combo   22
         chicken parmesan   21
       chicken parmigiana   18
           mac and cheese   18
               portobello   18
                 pastrami   16
           chicken cutlet   11
         steak and cheese    9
               roast beef    7
       f

Here is code to print the 10 best and 10 worst rated items. Look at the results, and decide whether you think it's important to consider the number of reviews when interpreting scores of which items are best and worst.

In [14]:
sorted_ratings = sorted(mean_ratings, key=mean_ratings.get)

print("Worst rated menu items:")
for item in sorted_ratings[:10]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")
    
print("\n\nBest rated menu items:")
for item in sorted_ratings[-10:]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")

Worst rated menu items:
chicken cutlet       Ave rating: 3.55 	count: 11
turkey sandwich      Ave rating: 3.80 	count: 5
spaghetti            Ave rating: 3.85 	count: 41
italian combo        Ave rating: 3.91 	count: 22
eggplant             Ave rating: 3.97 	count: 95
italian beef         Ave rating: 4.00 	count: 29
tuna salad           Ave rating: 4.00 	count: 5
garlic bread         Ave rating: 4.02 	count: 46
meatball             Ave rating: 4.08 	count: 163
portobello           Ave rating: 4.11 	count: 18


Best rated menu items:
prosciutto           Ave rating: 4.62 	count: 63
purista              Ave rating: 4.64 	count: 67
chicken salad        Ave rating: 4.67 	count: 6
pastrami             Ave rating: 4.69 	count: 16
reuben               Ave rating: 4.80 	count: 5
steak and cheese     Ave rating: 4.89 	count: 9
artichoke salad      Ave rating: 5.00 	count: 5
fettuccini alfredo   Ave rating: 5.00 	count: 6
turkey breast        Ave rating: 5.00 	count: 1
corned beef          Ave ra

Run the following line after you've decided your answer.

In [15]:
# check your answer (run this code cell to receive credit!)
q_5.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
    The less data you have for any specific item, the less you can trust that the average rating is the "real" sentiment of the customers. This is fairly common sense. If more people tell you the same thing, you're more likely to believe it. It's also mathematically sound. As the number of data points increases, the error on the mean decreases as 1 / sqrt(n).
    

In [16]:
# same output + errors
sorted_ratings = sorted(mean_ratings, key=mean_ratings.get)

print("Worst rated menu items:")
for item in sorted_ratings[:10]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} ± {1/(counts[item] ** 0.5):.2f} \tcount: {counts[item]}")
    
print("\n\nBest rated menu items:")
for item in sorted_ratings[-10:]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} ± {1/(counts[item] ** 0.5):.2f} \tcount: {counts[item]}")

Worst rated menu items:
chicken cutlet       Ave rating: 3.55 ± 0.30 	count: 11
turkey sandwich      Ave rating: 3.80 ± 0.45 	count: 5
spaghetti            Ave rating: 3.85 ± 0.16 	count: 41
italian combo        Ave rating: 3.91 ± 0.21 	count: 22
eggplant             Ave rating: 3.97 ± 0.10 	count: 95
italian beef         Ave rating: 4.00 ± 0.19 	count: 29
tuna salad           Ave rating: 4.00 ± 0.45 	count: 5
garlic bread         Ave rating: 4.02 ± 0.15 	count: 46
meatball             Ave rating: 4.08 ± 0.08 	count: 163
portobello           Ave rating: 4.11 ± 0.24 	count: 18


Best rated menu items:
prosciutto           Ave rating: 4.62 ± 0.13 	count: 63
purista              Ave rating: 4.64 ± 0.12 	count: 67
chicken salad        Ave rating: 4.67 ± 0.41 	count: 6
pastrami             Ave rating: 4.69 ± 0.25 	count: 16
reuben               Ave rating: 4.80 ± 0.45 	count: 5
steak and cheese     Ave rating: 4.89 ± 0.33 	count: 9
artichoke salad      Ave rating: 5.00 ± 0.45 	count: 5
fett

# Keep Going

Now that you are ready to combine your NLP skills with your ML skills, **[see how it's done](https://www.kaggle.com/matleonard/text-classification)**.

---
**[Natural Language Processing Home Page](https://www.kaggle.com/learn/natural-language-processing)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*